In [6]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pymorphy2
nltk.download('punkt')
nltk.download('stopwords')
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))


#данные названия анимешек 
data = {
    "title": [
        "Хантер x Хантер",
        "Токийский гуль",
        "Паразит",
        "Человек-бензопила",
        "Магическая битва",
        "Стальной алхимик",
        "Черный клевер",
        "Блич",
        "Атака титанов",
        "Наруто",
        "Город, в котором меня нет",
        "Клинок, рассекающий демонов",
        "Форма голоса",
        "Твое имя",
        "Хоримия",
        "Корзинка фруктов",
        "Неудержимая юность",
        "Очень приятно, Бог",
        "Тетрадь смерти",
        "Вайолет Эвергарден",
        "Убийца Акаме!",
        "Бездомный Бог",
        "Торадора!",
        "Последний серафим",
        "Токийские мстители"
    ],
    # данные синопсисы анимешек
    "synopsis": [
        "Охотник — это тот, кто путешествует по миру, выполняя различные опасные миссии: от поимки преступников до поисков сокровищ в неизведанных землях. Главный герой — мальчик по имени Гон. Его отец, Джин, был охотником, но исчез много лет назад. Гон считает, что если пойдёт по стопам отца и станет охотником, то рано или поздно вновь встретится с ним. Мальчик надеется, что, повстречав отца, наконец сможет задать ему один-единственный вопрос: почему он предпочёл жизнь охотника своему маленькому сынишке. Гон, как в своё время и его отец, покидает родной остров с целью сдать официальный экзамен на охотника. Но экзамен очень сложен, и каждый год множество людей погибают, пытаясь получить это звание. Во время своего путешествия Гон знакомится с Курапикой, Леорио и Киллуа, вместе с которыми ступает на тернистый путь охотника.",
        "Кэн Канэки — студент университета, который в результате нападения гуля попадает в больницу, где ему незаконно пересаживают органы этого же и одного из сильнейших гулей, чтобы спасти ему жизнь. Для того, чтобы выжить, гулям необходимо питаться человеческой плотью, поэтому они убивают людей или находят тела самоубийц. Из-за такой пересадки органов Канэки становится лишь наполовину гулем, но питаться человеческой плотью ему необходимо, как и всем остальным. Канэки стремится сохранить свою человечность, стараясь сохранить связь с миром людей, погрузившись в сообщество гулей.",
        "Жители одного японского города дождались контакта с инопланетным разумом. Но нельзя сказать, что людям их визит принёс пользу. Существа, по своей сути являющиеся паразитами, проникают в человеческий мозг через уши или нос и захватывают контроль над телом. Одному пареньку несказанно повезло: поскольку спал он в наушниках, пришелец не смог захватить мозг Синити Идзуми и ограничился рукой. Юноше удалось сохранить собственную личность, но теперь он не является единственным хозяином своего тела. Но так или иначе, у него есть возможность противостоять нашествию паразитов. Вступив в противостояние, у двух разумных существ, обитающих в одной телесной оболочке, появляется общая цель: остаться в живых. И для её достижения Синити и его неудачливому захватчику придётся заключить временное перемирие.",
        "У Дэндзи есть мечта — жить мирной и счастливой жизнью, проводя время с любимой девушкой. Однако суровая реальность в лице некоего якудзы заставляет парня вернуться с небес на землю, ведь он вынужден убивать демонов, чтобы расплатиться с необъятными долгами покойного отца. Вместе со своим дьявольским питомцем Почитой, которого использует в качестве оружия, Дэндзи готов сделать что угодно, даже за небольшую плату. Вскоре парень изжил свою полезность и был убит демоном, заключившим контракт с якудзой. Но верный Почита спасает своего хозяина, заключив с ним контракт. Почита стал одним целым с умирающим Дэндзи, заменив ему сердце, и подарил ему силы человека-бензопилы. Теперь переродившийся Дэндзи, получив умение превращать части своего тела в бензопилы, использует новообретённые способности, чтобы быстро и жестоко расправляться с врагами. Попав под внимание Бюро общественной безопасности, парень получает работу охотника на демонов. Отныне, не страшась даже самых жестоких противников, Дэндзи не остановится ни перед чем, чтобы воплотить в жизнь свои мечты.",
        "Действие аниме происходит в мире, где люди оказались вовсе не последним звеном в пищевой цепочке и любой ничего не подозревающий обыватель при известной доле невезения может оказаться съеденным проклятием. Пускай люди в большинстве своём не знают о существовании проклятий, те имеют уже давнюю историю, одним из эпизодов которой являются деяния легендарного демона Рёмэна Сукуны, которого с большим трудом удалось одолеть. Его тело было поделено на части, которые оказались разбросаны по миру. Если найдётся проклятие, способное поглотить все фрагменты его плоти, то оно обретёт мощь, способную уничтожить человеческую цивилизацию. Вот такая сложилась ситуация в мире, где в полном неведении о нависшей над человечеством угрозе проживает старшеклассник Юдзи Итадори. Но у Юдзи совершенно иные заботы. Он каждый день навещает в больнице дедушку и отнекивается от предложений спортивных кружков, которые мечтают заполучить его из-за выдающейся физической силы и выносливости. Однако Юдзи предпочёл вступить в клуб оккультных наук, с которого и начинаются все неприятности, когда в руки членов клуба попадает предмет, на который наложено проклятье. Не долго думая, школьники разбивают запечатывающее заклинание и выпускают на свет ужасные силы…",
        "Они дерзко нарушили самый главный закон в алхимии и очень сильно за это поплатились. Теперь двум братьям героям нужно найти загадочный и мистический философский камень. Вместе ребята отправляются на его поиски по всему миру, ведь только этот камень способен исправить то, что они сотворили... А ведь это не обычный мир, вместо науки тут властвует магия, и люди, обладающие способностью подчинять себе стихии. Магия тоже имеет свои ограничения, которым необходимо неукоснительно следовать. Тех, кто ослушается, ждет очень жестокая и страшная расправа. Братья Альфонс и Эдвард хотят совершить совершенно невообразимое и крайне запретное: воскресить из мертвых свою мать. Однако закон всемирного баланса не обмануть, если тебе нужно получить что-то, ты непременно должен пожертвовать чем-то равноценным.",
        "Ещё детьми Аста и Юно были оставлены у порога церкви в глухой деревушке. Мальчики пообещали друг другу, что один из них станет Королём магов. Однако чем старше они становились, тем больше различий между ними появлялось. Юно был гением магии, а Аста тренировал своё тело лишь физически. Когда им исполнилось по 15 лет, пришло время получать гримуары. Юно стал обладателем книги с четырёхлистным клевером (большинство людей получают книги с трёхлистным), в то время как Аста не получил ничего. Однако, когда Юно угрожала опасность, Аста раскрыл свою истинную силу. В итоге парень стал обладателем гримуара с пятилистным клевером под названием «Чёрный клевер»! Теперь два друга путешествуют по миру, чтобы достичь одну и ту же цель.",
        "Куросаки Ичиго — не простой пятнадцатилетний парень, потому как, сколько он себя помнил, всегда мог запросто общаться с духами и призраками, невидимыми для обычных людей. В один судьбоносный день ему является сама синигами (богиня смерти) Кучики Рукия, охотящаяся за Пустым (злым духом, пожирающим души людей). Во время битвы с этим чудовищем Ичиго и Рукия получили серьёзные ранения, и богине пришлось передать парню свои силы, чтобы помочь ему одержать победу над монстром, в результате чего Ичиго сам стал синигами! Но сделав это, Рукия теряет всю свою мощь, и теперь ожидает, что новоявленный синигами продолжит её миссию по отлову и уничтожению Пустых. Так начинаются увлекательные приключения Ичиго и Рукии.",
        "Однажды люди столкнулись с неразумными монстрами — титанами — и цивилизации пришел конец. Остатки человечества укрылись за тремя высокими стенами и приготовились встречать гигантов с особым оружием. Но укрепления надежно защитили выживших, и те привыкли к спокойному существованию в изоляции. Относительный мир продлился сотню лет. В один момент на город у внешней стены напали два разумных титана. Они выломали ворота и пустили внутрь своих собратьев. Город пал, а главный герой, мальчик Эрен Йегер, потерял мать. Эрен поклялся уничтожить всех титанов, чего бы это ни стоило. Вместе с другом Армином и приемной сестрой Микасой он вступил в армию и начал готовиться дать бой монстрам.",
        "За двенадцать лет до начала основных событий, Девятихвостый атаковал Деревню Скрытого Листа, уничтожив бóльшую часть деревни и унеся множество жизней. Лидер деревни, Четвёртый Хокаге, принёс себя в жертву, чтобы запечатать Девятихвостого в новорождённом мальчике — Наруто Узумаки. Осиротевшего, по причине нападения, Наруто избегали жители деревни, которые из чувства страха и злости видели в нём самого Девятихвостого. Несмотря на то, что Третий Хокаге запретил говорить обо всём, что касалось Девятихвостого, дети ментально перенимали свою неприязнь от родителей, и, тем самым, унаследовали враждебность по отношению к Наруто. Желая быть признанным, Наруто поклялся однажды стать величайшим Хокаге, которого видела деревня.",
        "История повествует о Сатору Фудзинумe, человеке, обладающем способностью возвращаться в прошлое за пару минут до рокового события, что позволяет ему предотвратить его. Когда убивают его мать, способность Сатору внезапно отправляет его на восемнадцать лет назад, когда он еще учился в начальной школе, давая ему возможность предотвратить похищение, унесшее жизни трех его сверстников.",
        "Ещё с древних времён ходят слухи о демонах-людоедах, что скрываются в лесах. Из-за чего местные жители боятся выходить на улицу ночью, дабы не пасть их жертвой. Но легенда гласит, что существует убийца демонов, который также рыщет под ночным небом, истребляя кровожадных демонов. Для одного мальчика эти слухи скоро станут суровой реальностью...Танджиро Камадо — старший сын в семье, потерявший своего отца и взявший на себя заботу о своих родных. Однажды он уходит в соседний город, чтобы продать древесный уголь. Вернувшись утром, парень обнаруживает, что вся его родня была зверски убита, а единственной выжившей является Нэзуко — обращённая в демона, но пока не потерявшая всю человечность младшая сестра. С этого момента для Танджиро и Нэзуко начинается долгое и опасное путешествие, в котором мальчик намерен узнать способ исцеления для своей сестры и разыскать убийцу.",
        "Детский мир жесток, прямолинеен и порой очень несправедлив, поскольку не терпит каких-либо различий. Тем не менее, все дети вырастают, но немногие из них способны, оглянувшись назад, признать свои ошибки. Кто-то может об этом даже не вспомнить, а кто-то - возненавидеть себя. В центре событий задира и настоящий разгильдяй Сёя, который в начальной школе ради веселья и обретения популярности среди приятелей превратил жизнь глухой одноклассницы Сёко в настоящий ад. Прошли годы, и теперь юноша осознал все свои ошибки, на собственном одиночестве и страданиях близких проникся историей прошлого и понял, насколько мерзкими были его поступки и сам он. Однажды уже изменившийся Сёя встречает Сёко, узнаёт её получше и понимает, что для того, чтобы быть хорошим человеком не обязательно походить на кого-то другого, а различия и вовсе делают нас особенными. Дальше парню предстоит выучить главный урок его жизни - чтобы сказать нечто важное кому-то, кто тебя не слышит, не обязательно использовать голос.",
        "Мицуха Миямидзу — обычная девушка, уставшая от жизни в провиннции. Её отец, мэр города, ведёт избирательную кампанию, а в семейном синтоистском храме ей приходится прилюдно исполнять древние ритуалы. И она мечтает перебраться из тесного провинциального мирка в большой город. Таки Татибана — увлекающийся архитектурой старшеклассник, вынужденный работать на полставки, чтобы обеспечивать свою жизнь в Токио. Изнурённый темпом большого города, Татибана мечтает о беззаботной жизни где-нибудь в горах. Однажды герои обнаруживают, что между ними существует странная и необъяснимая связь. Во сне они меняются телами и проживают жизни друг друга. Но однажды эта способность исчезает так же внезапно, как появилась. Таки решает во что бы то ни стало отыскать Мицуху, но способны ли они узнать друг друга в реальной жизни?",
        "В школе все одноклассники восхищались её дружелюбным отношением и успеваемостью, однако старшеклассница Кёко Хори скрывала свою другую сторону. Поскольку её родители часто заняты на работе, Хори всё время приходилось ухаживать за своим младшим братом и делать работу по дому, не оставляя возможности заводить друзей и общаться вне школы. У Изуми Миямуры, одноклассника Хори, тоже есть другая сторона. Его считают безнадежным отаку в очках, но на самом деле он мягкий человек и плох в учебе. У него девять пирсингов, скрытых за его длинными волосами и несколько татуировок. Случайно Хори и Миямура сталкиваются за пределами школы, ни один из них не выглядит так, как его считают другие. Эти двое, казалось бы, разные противоположности становятся верными друзьями и открывают ту сторону себя, которую они никогда никому не показывали.",
        "Жизнь старшеклассницы Тору Хонды переворачивается с ног на голову, когда её мама погибает, и она не просто остаётся одна в этом мире, но даже и без крыши над головой. Проблему она решает кардинально – переезжает в палатку в близлежащий лес. Вот только девушку продолжают преследовать неудачи, и оказывается, что она вторглась на частную территорию. Хозяева быстро обнаруживают нарушительницу, и Тору узнаёт, что земля принадлежит загадочной семье Сома. Тут чёрная полоса в жизни Тору вроде бы заканчивается, так как семья Сома предлагает поселиться у них. Вот только не слишком ли рано Тору обрадовалась? Действительно ли это везение или очередная западня судьбы? Тору ведь ещё не знает, что семья Сома проклята. Каждый её член превращается в одно из животных китайского зодиака, когда его обнимает человек противоположного пола.",
        "У Футабы Ёсиоки есть несколько причин, по которым в старшей школе она хочет начать свою жизнь с чистого листа. В средней школе из-за её красоты одноклассницы заставили девушку чувствовать себя изгоем. К тому же, из-за недопонимания она не смогла признаться в своих чувствах парню, которого она любила, Танаке. Теперь, в старшей школе, Футаба собирается вести себя как можно более неженственно, чтобы её друзья не ревновали к её внешности. В то время как она успешно живёт в таком образе, она снова встречает Танаку, который теперь носит имя Ко Мабути. Он говорит ей, что тоже испытывал к ней особые чувства, когда они были в средней школе, но сейчас всё не может быть как раньше. Сможет ли Футаба сохранить свою любовь, так и не начавшуюся три года назад?",
        "Момодзоно Нанами осталась с одним маленьким чемоданом на улице после того, как отца-должника лишили имущества. Брошенная и бездомная, она случайно помогает незнакомцу, который предлагает ей пожить в его доме, пока он будет отсутствовать. Каково же её удивление, когда «домом» оказывается заброшенный храм, населённый духами! А главный хранитель — лис Томоэ — явно не в восторге от новой «смотрительницы»...",
        "По сюжету старшекласснику Лайту Ягами в руки попадает тетрадь шинигами Рюка. Каждый человек, чьё имя записать в эту тетрадку, умрёт, поэтому Лайт решает с её помощью бороться со злом на земле.",
        "Некогда один известный профессор в помощь своей слепнущей жене-писательнице создал человекоподобную машину, так называемую автозапоминающую куклу. Подобные куклы получили широкое распространение, и вскоре так стали называть сотрудниц почты, которые помогают людям выражать чувства, находя нужные слова и обращая их в письма. Девушка Вайолет, потерявшая на войне обе руки, восхищается работой автозапоминающих кукол. Больше всего на свете она хочет стать одной из них, чтобы научиться понимать эмоции и узнать значение когда-то сказанных ей слов.",
        "Мечник Тацуми, простой парнишка из сельской местности, отправляется в столицу, чтобы заработать денег для своей голодающей деревни. По прибытии он узнает, что великая и красивая столица — лишь видимость. Город погряз в коррупции, жестокости и беззаконии, а виной тому — премьер-министр, который правит страной из-за кулис.",
        "Малоизвестный Бог Ято не имеет собственного храма, но мечтает стать уважаемым божеством с миллионами верующих. Правда, Ято не особо стремится прилагать усилия для достижения данной цели. Однажды обычная школьница Хиёри спасает его от дорожной аварии ценой собственной жизни.",
        "Рюдзи Такасу в жизни крупно не повезло. Покойный папаша-мафиози наградил сына таким лицом и глазами, что стоит ему не так взглянуть – учителя впадают в ступор, а одноклассники сами достают бумажники. В итоге Рюдзи, чье имя к тому же значит «Дракон», считается самым крутым хулиганом в округе и все его избегают, кроме единственного друга, Юсаку Китамуры. А ведь в душе главный герой – мягкий и романтичный парень, который еще и тащит на себе все домашнее хозяйство, вместе с попугаем Инко и непутевой маманей впридачу. Единственная отрада Рюдзи в том, что в старшей школе в один класс с ним попали не только лучший друг Китамура, но и любовь всей жизни Минору Кусиэда – милая и энергичная девушка. А теперь плохие новости: все, кроме него, уже в курсе, что туда же приходит маленькая, плоскогрудая и на редкость вспыльчивая красавица Тайга Айсака по прозвищу «Карманный Тигр»! Вся школа застыла в ожидании – что будет, когда столкнутся две местные легенды – Тигр и Дракон?! Чтобы узнать ответ, смотрите новую искрометную школьную комедию положений, снятую по бесспорным мотивам «Сяны», «Хаятэ» и «Луизы»!",
        "В стремлении к запретным знаниям «человек разумный» опять напортачил – в 2012 году на волю вырвался смертельный вирус, начавший косить человечество. Умирали почему-то взрослые, и тут, откуда ни возьмись, явились вампиры и согнали оставшихся детей в свой подземный город, объявив, что отныне «людишки» будут жить под защитой клыкастых хозяев, расплачиваясь кровью. Юитиро пожил так 4 года и решил бежать – вместе с названным братом Микой и всей «семьей Хякуя», группой детей, выросшей в одном приюте. Конечно, вампиры легко догнали и перебили беглецов, ушёл лишь один. Выйдя на поверхность, юный Хякуя понял, что эпидемия кончилась, но мир изменился – кругом разруха, бродят демоны, а кровососы потихоньку расширяют территорию, предлагая жителям «защиту». Власть в Японии захватила семья Хиираги, сформировавшая Имперскую Демоническую армию, которая защищает людей и бьет вампиров зачарованным оружием. Свой путь Юитиро ясен – готовиться к службе, мстить за погибшую семью. Прямого и порывистого парня ждет немало сюрпризов и неожиданностей, но в таком поганом мире лучше надеяться на свой дар и свой меч, чем на волю судьбы и благосклонность правителей!",
        "Жизнь 26-летнего Такемичи Ханагаки шла под откос. Он думал, что хуже быть не может, но затем внезапно узнаёт, что Хината Тачибана, его бывшая девушка, была убита «Свастонами»: группой злобных преступников, которые уже долгое время нарушают покой в обществе. Перед «смертью» он вспоминал лишь о своей первой любви, но, открыв глаза, внезапно понял, что переместился на 12-ть лет назад, во времена средней школы. Понимая, что у него есть шанс спасти Хинату, свою самую первую и единственную девушку, Такемичи решает проникнуть в «Свастоны» и подняться по «карьерной лестнице», чтобы переписать будущее и спасти Хинату от трагической судьбы."
    ]
}
# жанры анимешек
genres = {
        "Хантер x Хантер": "экшен приключения сёнэн", 
        "Токийский гуль": "экшен тёмный драма психология ужасы", 
        "Паразит": "ужасы драма философия фантастика",
        "Человек-бензопила": "экшен тёмный ужасы сёнэн",
        "Магическая битва": "экшен магия сёнэн",
        "Стальной алхимик": "экшен магия драма война философия",
        "Черный клевер": "экшен магия сёнэн",
        "Блич": "экшен сверхъестественное сёнэн",
        "Атака титанов": "экшен война драма война постапокалипсис",
        "Наруто": "ниндзя приключения дружба",
        "Город, в котором меня нет": "драма психология триллер",
        "Клинок, рассекающий демонов": "экшен тёмный демоны сёнэн",
        "Форма голоса": "драма школа драма психология",
        "Твое имя": "романтика фантастика драма романтика",
        "Хоримия": "романтика школа повседневность",
        "Корзинка фруктов": "драма семья романтика школа",
        "Неудержимая юность": "романтика школа",
        "Очень приятно, Бог": "романтика сверхъестественное",
        "Тетрадь смерти": "драма психология триллер сверхъестественное",
        "Вайолет Эвергарден": "драма эмоции повседневность",
        "Убийца Акаме!": "экшен тёмный драма война",
        "Бездомный Бог": "экшен сверхъестественное комедия",
        "Торадора!": "романтика школа комедия",
        "Последний серафим": "экшен тёмный вампиры постапокалипсис",
        "Токийские мстители": "драма криминал экшен боевка" 
}
# типы анимешек
types = {
        "Хантер x Хантер": "приключенческий_экшен",
        "Токийский гуль": "тёмный_хоррор",
        "Паразит": "тёмная_фантастика",
        "Человек-бензопила": "тёмный_экшен",
        "Магическая битва": "боевой_сёнэн",
        "Стальной алхимик": "эпический_экшен",
        "Черный клевер": "боевой_сёнэн",
        "Блич": "боевой_сёнэн",
        "Атака титанов": "эпическая_трагедия",
        "Наруто": "приключенческий_сёнэн",
        "Город, в котором меня нет": "психологическая_драма",
        "Клинок, рассекающий демонов": "боевой_сёнэн",
        "Форма голоса": "повседневная_драма",
        "Твое имя": "романтическая_драма",
        "Хоримия": "школьная_романтика",
        "Корзинка фруктов": "семейная_драма",
        "Неудержимая юность": "школьная_романтика",
        "Очень приятно, Бог": "романтическое_фэнтези",
        "Тетрадь смерти": "психологический_триллер",
        "Вайолет Эвергарден": "эмоциональная_драма",
        "Убийца Акаме!": "трагический_экшен",
        "Бездомный Бог": "приключенческое_фэнтези",
        "Торадора!": "романтическая_комедия",
        "Последний серафим": "постапокалиптический_экшен",
        "Токийские мстители": "криминальная_драма"
}

df = pd.DataFrame(data)
df["genre_text"] = df["title"].map(genres)
df["type_text"] = df["title"].map(types)


#Препроцессинг синопсисов
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^а-яё\s]', ' ', text)
    tokens = word_tokenize(text, language='russian')
    tokens = [
        morph.parse(token)[0].normal_form
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]

    return ' '.join(tokens)

df['preprocessed_synopsis'] = df['synopsis'].apply(preprocess)

# Преобразование текста в чис. векторы
plot_tfidf = TfidfVectorizer().fit_transform(df["preprocessed_synopsis"])
genre_tfidf = TfidfVectorizer().fit_transform(df["genre_text"])
type_tfidf = TfidfVectorizer().fit_transform(df["type_text"])

# Косинусная схожесть
plot_sim = cosine_similarity(plot_tfidf)
genre_sim = cosine_similarity(genre_tfidf)
type_sim = cosine_similarity(type_tfidf)

final_sim = 0.3 * plot_sim + 0.4 * genre_sim + 0.3 * type_sim

# Делаем рекомендательную системку
def anime_recommend(title, top_n=5):
    idx = df.index[df["title"] == title][0]
    scores = pd.Series(final_sim[idx], index=df["title"]).drop(title)
    return scores.sort_values(ascending=False).head(top_n)
# Просто чтоб красивенько было
def recommend_result(title, top_n=5):
    recs = anime_recommend(title, top_n)
    if isinstance(recs, str):
        print(recs)
        return

    print(f"Рекомендации по: {title}\n")
    for i, (anime, score) in enumerate(recs.items(), 1):
        print(f"{i}. {anime:<30} — {score*100:.2f}%")

recommend_result("Человек-бензопила")

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


Рекомендации по: Человек-бензопила

1. Токийский гуль                 — 28.44%
2. Клинок, рассекающий демонов    — 27.02%
3. Блич                           — 18.87%
4. Хантер x Хантер                — 18.85%
5. Черный клевер                  — 18.80%
